In [2]:
from copy import deepcopy

from aiida.orm import Bool, Int
from IPython.core.interactiveshell import InteractiveShell
from pymatgen.io.ase import AseAtomsAdaptor

InteractiveShell.ast_node_interactivity = "all"

import json
import os
import re
from copy import deepcopy
from pathlib import Path

from aiida import load_profile
from aiida.engine import submit
from aiida.orm import Bool, Dict, Float, Int, StructureData, load_code, load_node
from aiida.orm.nodes.data.structure import Site
from aiida.plugins import DataFactory
from aiida_quantumespresso.common.types import SpinType
from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData
from aiida_quantumespresso_hp.workflows.hubbard import SelfConsistentHubbardWorkChain
from ase.atoms import Atoms
from ase.visualize import view
from pyconfsamp.core import change_atom_names

load_profile()
from pprint import pprint

import numpy as np
import pandas as pd
from project_settings import project_dir
from sklearn.metrics import mean_absolute_error as mae

In [3]:
fully_lithiated_df = pd.read_pickle(os.path.join(project_dir, 'data', 'fully_lithiated_df.pkl'))
mn_olivine_ase = fully_lithiated_df['ase_in'].values[2]

sorting_dict = {
    'Mn': 0,
    'O': 1,
    'P': 2,
    'Li': 3,
}

# ! Does not take into account initial magmoms for now
mn_olivine_ase_ordered = Atoms(sorted(mn_olivine_ase, key=lambda x: sorting_dict[x.symbol]))
mn_olivine_ase_ordered.set_cell(mn_olivine_ase.get_cell())
mn_olivine_ase_ordered.center()
mn_olivine_ase_ordered.symbols

lmpo_structure_data = StructureData(ase=mn_olivine_ase_ordered)

Symbols('Mn4O16P4Li4')

In [4]:

# ! Currently, for whatever reason, the codes are duplicated
# pw_code = load_code('qe-dev-pw@lumi-small')
# hp_code = load_code('qe-dev-hp@lumi-small')
pw_code = load_code(2182)
hp_code = load_code(2183)

hubbard_data = load_node(3075)
print(hubbard_data.get_quantum_espresso_hubbard_card())

HUBBARD	ortho-atomic
 U	Mn-3d	4.5618
 V	Mn-3d	O-2p	1	518	0.0001
 V	Mn-3d	O-2p	1	601	0.0001
 V	Mn-3d	O-2p	1	373	0.0001
 V	Mn-3d	O-2p	1	621	0.0001
 V	Mn-3d	O-2p	1	326	0.0001
 V	Mn-3d	O-2p	1	17	0.0001
 V	Mn-3d	O-2p	1	596	0.0001
 V	Mn-3d	O-2p	2	15	0.0001
 V	Mn-3d	O-2p	2	324	0.0001
 V	Mn-3d	O-2p	2	12	0.0001
 V	Mn-3d	O-2p	2	8	0.0001
 V	Mn-3d	O-2p	2	19	0.0001
 V	Mn-3d	O-2p	2	328	0.0001
 V	Mn-3d	O-2p	2	5	0.0001
 V	Mn-3d	O-2p	3	18	0.0001
 V	Mn-3d	O-2p	3	17	0.0001
 V	Mn-3d	O-2p	3	11	0.0001
 V	Mn-3d	O-2p	3	371	0.0001
 V	Mn-3d	O-2p	3	378	0.0001
 V	Mn-3d	O-2p	3	377	0.0001
 V	Mn-3d	O-2p	3	6	0.0001
 V	Mn-3d	O-2p	4	20	0.0001
 V	Mn-3d	O-2p	4	19	0.0001
 V	Mn-3d	O-2p	4	626	0.0001
 V	Mn-3d	O-2p	4	6	0.0001
 V	Mn-3d	O-2p	4	632	0.0001
 V	Mn-3d	O-2p	4	631	0.0001
 V	Mn-3d	O-2p	4	7	0.0001



***
## Set up workchain with all of Iurii's settings

In [ ]:
afm_builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'olivine_iurii_overrides.yaml')),
    spin_type=SpinType.COLLINEAR,
)

scf_dict = afm_builder.scf.pw.parameters.get_dict()
relax_dict = afm_builder.relax.base.pw.parameters.get_dict()
hubbard_dict = afm_builder.hubbard.hp.parameters.get_dict()

# print(json.dumps(scf_dict, sort_keys=False, indent=4))
# print(json.dumps(relax_dict, sort_keys=False, indent=4))
# print(json.dumps(hubbard_dict, sort_keys=False, indent=4))

In [ ]:
# proper_olivine_builder = deepcopy(builder)
# proper_olivine_submit = submit(proper_olivine_builder)
# proper_olivine_submit_pk = proper_olivine_submit.pk
proper_olivine_submit_pk = 4282

In [ ]:
!verdi process status 4282

***
## Change only force convergence threshold in the optimization to see if `reconstruction problem` persists

In [ ]:
only_force_thr_builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'only_force_thr.yaml')),
    spin_type=SpinType.COLLINEAR,
)

# only_force_thr_builder = submit(only_force_thr_builder)
# only_force_thr_submit_pk = only_force_thr_builder.pk
# print(only_force_thr_submit_pk)

In [ ]:
# only_force_thr_submit_pk = 4376
!verdi process status 4376

***
## True single-shot (only scf + hp), with default workchain settings

In [ ]:
true_singleshot_builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'basically_empty_overrides.yaml')),
    spin_type=SpinType.COLLINEAR,
)

_ = true_singleshot_builder.pop("relax", None)
true_singleshot_builder.meta_convergence = Bool(False)
true_singleshot_builder.max_iterations = Int(1)

# true_singleshot_submit = submit(true_singleshot_builder)
# print(true_singleshot_submit.pk)



In [ ]:
true_singleshot_submit_pk = 4774
# !verdi process status 4774
true_ss_workchain_node = load_node(true_singleshot_submit_pk)
print(true_ss_workchain_node.mtime - true_ss_workchain_node.ctime)

***
## True single-shot with U=0 as starting value

In [5]:
u0_hubbard_data = HubbardStructureData(lmpo_structure_data)
u0_hubbard_data.initialize_onsites_hubbard('Mn', '3d', 0.0)
u0_hubbard_data.initialize_intersites_hubbard('Mn', '3d', 'O', '2p', 0.0001, number_of_neighbours=7) 
# print(u0_hubbard_data.get_quantum_espresso_hubbard_card())

true_ss_u0_builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=u0_hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'basically_empty_overrides.yaml')),
    spin_type=SpinType.COLLINEAR,
)

_ = true_ss_u0_builder.pop("relax", None)
true_ss_u0_builder.meta_convergence = Bool(False)
true_ss_u0_builder.max_iterations = Int(1)

true_ss_u0_submit = submit(true_ss_u0_builder)
print(true_ss_u0_submit.pk)

5019


In [ ]:
# true_ss_u0_submit_pk = 5019
# # !verdi process status 5019
# true_ss_workchain_node = load_node(true_ss_u0_submit_pk)
# print(true_ss_workchain_node.mtime - true_ss_workchain_node.ctime)

***
## Default workchain settings, no relaxation


In [ ]:
default_norelax_builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'basically_empty_overrides.yaml')),
    spin_type=SpinType.COLLINEAR,
)

# _ = default_norelax_builder.pop("relax", None)
# default_norelax_submit = submit(default_norelax_builder)
# print(default_norelax_submit.pk)


In [ ]:
default_norelax_submit_pk = 4512
# !verdi process status 4512
default_workchain_node = load_node(default_norelax_submit_pk)
print(default_workchain_node.mtime - default_workchain_node.ctime)

***
## All of Iurii's settings, but no relaxation

In [ ]:
iurii_norelax_builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'olivine_iurii_overrides.yaml')),
    spin_type=SpinType.COLLINEAR,
)

_ = iurii_norelax_builder.pop("relax", None)
# iurii_norelax_builder = submit(iurii_norelax_builder)
# print(iurii_norelax_builder.pk)

In [ ]:
iurii_norelax_builder_pk = 4486 
# !verdi process status 4486
iurii_workchain_node = load_node(iurii_norelax_builder_pk)
print(iurii_workchain_node.mtime - iurii_workchain_node.ctime)

***
## Compare results of true SS, workchain defaults, and Iurii's settings

In [ ]:
true_ss_output_hubbard = load_node(4813)
default_output_hubbard = load_node(4699)
iurii_output_hubbard = load_node(4822)

In [ ]:
# type(true_ss_output_hubbard)

true_ss_hubbard_array = np.array([el[4] for el in true_ss_output_hubbard.hubbard_parameters])
default_hubbard_array = np.array([el[4] for el in default_output_hubbard.hubbard_parameters])
iurii_hubbard_array = np.array([el[4] for el in iurii_output_hubbard.hubbard_parameters])

pprint(mae(true_ss_hubbard_array, default_hubbard_array))
pprint(mae(true_ss_hubbard_array, iurii_hubbard_array))
pprint(mae(default_hubbard_array, iurii_hubbard_array))

print(true_ss_output_hubbard.get_quantum_espresso_hubbard_card())
print(default_output_hubbard.get_quantum_espresso_hubbard_card())
print(iurii_output_hubbard.get_quantum_espresso_hubbard_card())

pprint(true_ss_hubbard_array)
pprint(default_hubbard_array)
pprint(iurii_hubbard_array)

print(np.abs(true_ss_hubbard_array-iurii_hubbard_array))
print(np.abs(default_hubbard_array-iurii_hubbard_array))


In [ ]:
pprint(true_ss_hubbard_array-default_hubbard_array)
pprint(default_hubbard_array-iurii_hubbard_array)

***
## Set proper AFM magnetic ordering
### Reorder the structure such that Hubbard atoms come first (required by `hp.x`)

*** 
### Make Mn atoms distinct for providing initial AFM ordering

In [60]:
relabel_dict = {
    0: "Mn1",
    1: "Mn1",
    2: "Mn2",
    3: "Mn2",
}

afm_structure_data = change_atom_names(
    structure_data=lmpo_structure_data, relabel_dict=relabel_dict
)
afm_structure_data.get_composition()
afm_structure_data.get_kind_names()

{'P': 4, 'O': 16, 'Mn': 4, 'Li': 4}

['Mn1', 'Mn2', 'O', 'P', 'Li']

In [66]:
# Set the initial U and V values for the distinct Mn1 and Mn2 sites
afm_hubbard_data = HubbardStructureData(structure=afm_structure_data)
# /home/jgeiger/projects/bat_uv_ml/data/olivine_iurii/LixMnPO4/Li1.00/DFT_plus_UV/9_PDOS/LMPO.scf.1.in
afm_hubbard_data.initialize_onsites_hubbard('Mn1', '3d', 4.5618)
afm_hubbard_data.initialize_onsites_hubbard('Mn2', '3d', 4.5618)
afm_hubbard_data.initialize_intersites_hubbard('Mn1', '3d', 'O', '2p', 0.0001, number_of_neighbours=7) 
afm_hubbard_data.initialize_intersites_hubbard('Mn2', '3d', 'O', '2p', 0.0001, number_of_neighbours=7) 

In [68]:
afm_ss_builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=afm_hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'basically_empty_overrides.yaml')),
)

_ = afm_ss_builder.pop("relax", None)
afm_ss_builder.meta_convergence = Bool(False)
afm_ss_builder.max_iterations = Int(1)

mag_dict = {
    "starting_magnetization": {
        "Mn1": 0.5,
        "Mn2": -0.5,
        "O": 0.0,
        "P": 0.0,
        "Li": 0.0,
    },
    "nspin": 2,
}

afm_ss_builder.scf.pw.parameters["SYSTEM"] = mag_dict

afm_ss_submit = submit(afm_ss_builder)
print(afm_ss_submit.pk)

4899


In [ ]:
afm_ss_submit_pk = 4899
# !verdi process status 4899
# afm_ss_workchain_node = load_node(afm_ss_submit_pk)
# print(afm_ss_workchain_node.mtime - afm_ss_workchain_node.ctime)